## Imports

In [23]:
from geopy.geocoders import GoogleV3
import pandas as pd
import numpy as np
import os
import geopandas as gpd

## Read-in

In [24]:
df = pd.read_csv('Hudson_yards_map_template_popup_pin - Sheet1.csv')

In [25]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('hudson_yards.kml',driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_10188/407052935.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  project_gdf['center_point'] = project_gdf.geometry.centroid


## Geocoder Setup

In [26]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [27]:
df['geo_address'] = df['Address'] + ' Manhattan, NY'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [34]:
# 40.75725152723692, -74.00053686779148
df.at[16,'lat']=()

,Address,sq_ft,developers,description,story_link,project name,floors,status,notable tenants,geo_address,loc,point,lat,lon,altitude
0,501 West 30th Street,1.8 million,Related Companies/Oxford Properties,Office building,NaN,10 Hudson Yards,52,Open,"Tapestry, SAP, L'Oreal USA","501 West 30th Street Manhattan, NY","(501 W 30th St, New York, NY 10001, USA, (40.7...","(40.7525462, -74.0011174, 0.0)",40.752546,-74.001117,0.0
1,500 West 33rd Street,2.6 million,Related Companies/Oxford Properties,Office/Retail building,NaN,30 Hudson Yards,100,Open,"Warner Media, Wells Fargo, KKR","500 West 33rd Street Manhattan, NY","(500 W 33rd St, New York, NY 10001, USA, (40.7...","(40.75388, -74.0000046, 0.0)",40.753880,-74.000005,0.0
2,427 10th Avenue,2.9 million,Related Companies/Oxford Properties,Office/Retail building,NaN,50 Hudson Yards,77,Open,"BlackRock, Meta, Truist Financial","427 10th Avenue Manhattan, NY","(427 10th Ave, New York, NY 10001, USA, (40.75...","(40.7544626, -73.9995398, 0.0)",40.754463,-73.999540,0.0
3,380 11th Avenue,1.3 million,Related Companies/Oxford Properties,Office/Retail building,NaN,55 Hudson Yards,51,Open,"Meta, Point72, Cooley","380 11th Avenue Manhattan, NY","(380 11th Ave, New York, NY 10001, USA, (40.75...","(40.7552049, -74.0020739, 0.0)",40.755205,-74.002074,0.0
4,514 West 36th Street,1.3 million,Related Companies/Oxford Properties,Office building,NaN,NaN,25 tentatively,In Development,NaN,"514 West 36th Street Manhattan, NY","(514 W 36th St., New York, NY 10018, USA, (40....","(40.7561471, -73.9991623, 0.0)",40.756147,-73.999162,0.0
5,400 11th Avenue,1.9 million,Boston Properties/The Moinian Group,Office/Retail building,NaN,3 Hudson Boulevard,56,In Development,NaN,"400 11th Avenue Manhattan, NY","(400 11th Ave, New York, NY 10001, USA, (40.75...","(40.7559316, -74.0011117, 0.0)",40.755932,-74.001112,0.0
6,527 West 34th Street/66 Hudson Blvd,2.8 million,Tishman Speyer,Office/Retail building,NaN,The Spiral,66,In Development,"HSBC, Pfizer, NewYork Presbyterian","527 West 34th Street/66 Hudson Blvd Manhattan, NY","(527 W 34th St. #66, New York, NY 10001, USA, ...","(40.7554969, -74.0006879, 0.0)",40.755497,-74.000688,0.0
7,395 9th Avenue,2.1 million,Brookfield Properties,Office/Retail building,NaN,One Manhattan West,67,Open,"EY, Skadden, NHL, Accenture","395 9th Avenue Manhattan, NY","(395 9th Ave, New York, NY 10001, USA, (40.752...","(40.7526117, -73.99737569999999, 0.0)",40.752612,-73.997376,0.0
8,435 West 31st Street,2.0 million,Brookfield Properties,Office/Retail building,NaN,Two Manhattan West,58,In Development,"D.E. Shaw, KPMG, Clifford Chance","435 West 31st Street Manhattan, NY","(435 W 31st St, New York, NY 10001, USA, (40.7...","(40.7522538, -73.9990079, 0.0)",40.752254,-73.999008,0.0
9,424/434 West 33rd Street,"184,000",Brookfield Properties,Office building,NaN,Four Manhattan West,13,Open,NaN,"424/434 West 33rd Street Manhattan, NY","(424-434 W 33rd St, New York, NY 10001, USA, (...","(40.7531432, -73.9981961, 0.0)",40.753143,-73.998196,0.0


## HTML Popup Formatter

In [28]:
df.columns

Index(['Address', 'sq_ft', 'developers', 'description', 'story_link',
       'project name', 'floors', 'status', 'notable tenants', 'geo_address',
       'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [29]:
def popup_html(row):
    i = row
    address = df['Address'].iloc[i]
    developers = df['developers'].iloc[i]
    description = df['description'].iloc[i]
    project_name = df['project name'].iloc[i]
    floors = df['floors'].iloc[i]
    status = df['status'].iloc[i]
    notable_tenants = df['notable tenants'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developers) + '''</strong><br>
    Description: <strong>{}'''.format(description) + '''</strong><br>
    Project Name: <strong>{}'''.format(project_name) + '''</strong><br>
    Floors: <strong>{}'''.format(floors) + '''</strong><br>
    Status: <strong>{}'''.format(status) + '''</strong><br>
    Notable Tenants: <strong>{}'''.format(notable_tenants) + '''</strong><br>
    </html>
    '''
    return html


In [30]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Hudson Yards Projects')

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=300, max_width=300)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [31]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [33]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/hudson_yards_map_Pat_Ralph
